In [1]:
import numpy as np
import pandas as pd
import torch.nn as nn
import PIL
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True #imagenes ligeramente corruptas OK
from transformers import AutoImageProcessor
from transformers import AutoModelForImageClassification

In [2]:
data=pd.read_feather("data/stage1.feather")

In [3]:
data["predicted"]=np.full(data.shape[0],"")

In [4]:
data["certainty"]=np.full(data.shape[0],"")

In [5]:
model_name="./models-legacy/swinv2-base-patch4-window16-256-retrained-f1-85/"

In [6]:
device="cuda:2"

In [7]:
model = AutoModelForImageClassification.from_pretrained(model_name).to(device)

In [8]:
image_processor = AutoImageProcessor.from_pretrained(model_name)

In [9]:
def load_image(path):
    return PIL.Image.open(path)

In [10]:
def process(image):
    return image_processor(images=image, return_tensors="pt").to(model.device)["pixel_values"]

In [11]:
def consume(pixels):
    logits=model(pixel_values=pixels).logits
    return nn.functional.softmax(logits, dim=-1).detach().cpu().numpy().flatten()

In [12]:
def classify(probabilities):
    idx=probabilities.argmax(-1).item()
    return model.config.id2label[idx]

In [13]:
from fastprogress import progress_bar as pb

In [14]:
for idx,r in pb(data.iterrows(), data.shape[0]):
    try:
        probs=consume(process(load_image(r["path"].replace("\\","/"))))
        cl=classify(probs)
        crt=probs.max()
    except Exception as E:
        print("ERROR:",E)
        print(r)
        cl="UNK"
        crt=0
    data.loc[idx,"predicted"]=cl
    data.loc[idx,"certainty"]=crt

ERROR: Unsupported number of image dimensions: 2
path         images/2001/jul/p0703012.jpg
class                                    
predicted                                
certainty                                
Name: 132, dtype: object
ERROR: Unable to infer channel dimension format
path         images/2012/may/p0522126.jpg
class                                    
predicted                                
certainty                                
Name: 927, dtype: object
ERROR: Unable to infer channel dimension format
path         images/2014/dic/p12081412.jpg
class                                     
predicted                                 
certainty                                 
Name: 3646, dtype: object
ERROR: Unsupported number of image dimensions: 2
path         images/2002/ene/p0123025.jpg
class                                    
predicted                                
certainty                                
Name: 4882, dtype: object
ERROR: Unsupported number of

In [19]:
data

,path,class,predicted,certainty
0,images/2001/ene/p0130012.jpg,,FUM,0.997546
1,images/2001/ene/p0201011.jpg,,FUM,0.999964
2,images/2001/ene/p0123015.jpg,,FUM,0.999991
3,images/2001/ene/p0126001.jpg,,FUM,0.99874
4,images/2001/ene/p0122012.jpg,,FUM,0.999987
...,...,...,...,...
20190,images/2015/ago/p0815153.jpg,,INA,0.738808
20191,images/2015/ago/p0802155.jpg,,EXP+FUM,0.993867
20192,images/2015/ago/p0807153.jpg,,FUM,0.999897
20193,images/2015/ago/p0805153.jpg,,FUM,0.999935


In [20]:
data.drop(data[(data["predicted"]=="UNK") & (data["certainty"]>=.999)].index, inplace=True)

In [21]:
data.shape

(19341, 4)

In [22]:
data.drop(data[data["certainty"]==0].index, inplace=True)

In [23]:
data.shape

(19288, 4)

In [24]:
data.sort_values(by=["predicted", "certainty"],inplace=True,ascending=False)

In [25]:
data.reset_index(drop=True, inplace=True)

In [26]:
data

,path,class,predicted,certainty
0,images/2006/jun/p0622063.jpg,,UNK,0.998998
1,images/2023/jul/p0704235.jpg,,UNK,0.998997
2,images/2013/sep/p0926132.jpg,,UNK,0.998995
3,images/2019/ene/p0120193.jpg,,UNK,0.998994
4,images/2018/jul/p0723181.jpg,,UNK,0.998969
...,...,...,...,...
19283,images/2015/may/p0530155.jpg,,EXP,0.489222
19284,images/2016/oct/p1028164.jpg,,EXP,0.461512
19285,images/2018/dic/p1231184.jpg,,EXP,0.459877
19286,images/2018/jun/p0603184.jpg,,EXP,0.421862


In [27]:
data.to_feather(f"data/stage2.feather")